In [5]:
!pip install google-cloud --user
!pip install --upgrade google-cloud-bigquery[pandas] --user
!pip install google-cloud-storage --user

DEPRECATION: Loading egg at /Users/azizamed/anaconda3/lib/python3.11/site-packages/pandas-1.5.3-py3.11-macosx-10.9-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/azizamed/anaconda3/lib/python3.11/site-packages/psycopg2_binary-2.9.9-py3.11-macosx-10.9-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/azizamed/anaconda3/lib/python3.11/site-packages/fonttools-4.46.0-py3.11-macosx-10.9-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /User

In [25]:
import logging
import json
import os
from datetime import datetime
import pprint

# Default scope to get access token
_SCOPE = 'https://www.googleapis.com/auth/cloud-platform'
from google.cloud import bigquery
client = bigquery.Client.from_service_account_json('../../data_warehousing/data_warehousing/include/gcp/service_account.json')

In [26]:
QUERY = (
    'SELECT * FROM `ariflow-tutorial.youtube.raw_cities` '
    'LIMIT 10')
query_job = client.query(QUERY)  # API request
rows = query_job.result()
df=rows.to_dataframe()

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Cities                 9 non-null      object 
 1   City_name              8 non-null      object 
 2   Geography              8 non-null      object 
 3   Geography_3            8 non-null      object 
 4   Views                  9 non-null      Int64  
 5   Watch_time__hours_     9 non-null      float64
 6   Average_view_duration  9 non-null      dbtime 
dtypes: Int64(1), dbtime(1), float64(1), object(4)
memory usage: 645.0+ bytes


In [24]:
df.describe

<bound method NDFrame.describe of          Date  TotalViews
0  2022-08-09         311
1  2023-12-01         191
2  2021-06-21         181
3  2022-09-26         367
4  2023-11-29         263
5  2022-09-25         179
6  2022-08-08         363
7  2023-11-28         359
8  2023-11-30         202
9  2023-11-27         332>

In [30]:
def get_table_names(client, dataset_id):
    """Return a list of table names in the specified dataset."""
    table_names = []
    dataset_ref = client.dataset(dataset_id)
    
    # List tables in the dataset
    tables = client.list_tables(dataset_ref)
    
    for table in tables:
        table_names.append(table.table_id)
    
    return table_names

In [32]:
get_table_names(client,'youtube')

['dim_dail_view_7day', 'dim_top10', 'raw_cities', 'raw_gender', 'raw_total']

In [33]:
def get_column_names(client, dataset_id, table_name):
    """Return a list of column names for the specified table."""
    column_names = []
    table_ref = client.dataset(dataset_id).table(table_name)
    table = client.get_table(table_ref)
    for field in table.schema:
        column_names.append(field.name)
    return column_names

In [35]:
get_column_names(client,'youtube','raw_cities')

['Cities',
 'City_name',
 'Geography',
 'Geography_3',
 'Views',
 'Watch_time__hours_',
 'Average_view_duration']

In [40]:
def get_database_info(client, dataset_id):
    """Return a list of dicts containing the table name and columns for each table in the dataset."""
    table_dicts = []
    for table_name in get_table_names(client, dataset_id):
        column_names = get_column_names(client, dataset_id, table_name)
        table_dicts.append({"table_name": table_name, "column_names": column_names})
    return table_dicts

In [41]:
get_database_info(client,'youtube')

[{'table_name': 'dim_dail_view_7day',
  'column_names': ['Date', 'RollingAverageViews']},
 {'table_name': 'dim_top10', 'column_names': ['Date', 'TotalViews']},
 {'table_name': 'raw_cities',
  'column_names': ['Cities',
   'City_name',
   'Geography',
   'Geography_3',
   'Views',
   'Watch_time__hours_',
   'Average_view_duration']},
 {'table_name': 'raw_gender',
  'column_names': ['Date',
   'Views',
   'Watch_time__hours_',
   'Average_view_duration']},
 {'table_name': 'raw_total', 'column_names': ['Date', 'Views']}]